In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy.ext.declarative import declared_attr
from sqlalchemy import Column, Integer, String, Enum, Date, Sequence, DateTime, Boolean, ForeignKey, Table, BLOB, ARRAY, ForeignKeyConstraint, Float

from sqlalchemy.orm.exc import NoResultFound
from sqlalchemy.orm import sessionmaker, relationship
import datetime
import hashlib, binascii, os

def hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'),
                                  salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')


engine = create_engine('sqlite:////home/morelle/DBs/test.db', echo=True)

In [2]:
Base = declarative_base()

In [3]:
class TimestampMixin(object):
    created_at = Column(DateTime, default=datetime.datetime.utcnow)
    
    
class TableNameMixin(object):
    @declared_attr
    def __tablename__(cls):
        return cls.__name__.lower()+'s'
    
class TableIdMixin(object):
    @declared_attr
    def id(cls):
        return Column(Integer, Sequence('{}_id_seq'.format(cls.__tablename__)), primary_key=True)
    
class TableNameIdMixin(TableNameMixin, TableIdMixin):
    pass

class PersonMixin(object):
    firstname = Column(String(50), nullable=False)
    lastname = Column(String(50), nullable=False)
    
class AnnotationMixin(TimestampMixin):
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    name = Column(String(20), nullable = False)

In [4]:
users_groups_associations = Table('users_groups_associations', Base.metadata,
    Column('users_id', Integer, ForeignKey('users.id')),
    Column('groups_id', Integer, ForeignKey('groups.id'))
)

class User(Base, TableNameIdMixin, PersonMixin):
    username = Column(String(20), nullable=False, unique=True)
    password_hash = Column(String(200), nullable=False)
    
    groups = relationship(
        "Group",
        secondary=users_groups_associations,
        back_populates="users")
    window_positions = relationship("WindowPosition", back_populates="user")
    
    
    
    def __repr__(self):
        return "<User(firstname={}, lastname={}, id={})>".format(
            self.firstname, self.lastname, self.id)
    
class WindowPosition(Base, TableNameIdMixin):    
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    user = relationship("User", back_populates="window_positions")
    window_type = Column(String, nullable=False) 
    
    top_left_x = Column(Integer, nullable=False)
    top_left_y = Column(Integer, nullable=False)
    
    bot_right_x = Column(Integer, nullable=False)
    bot_right_y = Column(Integer, nullable=False)
    
class Dataset_Group_Association(Base, TableNameMixin):
    group_id = Column(Integer, ForeignKey('groups.id'), primary_key=True)
    dataset_id = Column(Integer, ForeignKey('datasets.id'), primary_key=True)
    read = Column(Boolean, nullable=False, default=False)
    write = Column(Boolean, nullable=False, default=False)
    dlt = Column(Boolean, nullable=False, default=False)
    
    group = relationship("Group", back_populates="datasets")
    dataset = relationship("Dataset", back_populates="groups")
    
class Group(Base, TableNameIdMixin):
    name = Column(String(20), nullable=False)
    users = relationship(
        "User",
        secondary=users_groups_associations,
        back_populates="groups")
    
    datasets = relationship("Dataset_Group_Association", back_populates="group")
    patients = relationship("Patient", back_populates="groups")
    
class Dataset(Base, TableNameIdMixin):
    info = Column(String(10000), nullable=False)
    name = Column(String(50))
    
    groups = relationship("Dataset_Group_Association", back_populates = "dataset")
    collections = relationship("Collection", back_populates = "dataset")
    
shares_users_associations = Table('shares_users_associations', Base.metadata,
    Column('shares_user', Integer, primary_key=True),
    Column('shares_group', Integer, primary_key=True),
    Column('shares_dataset', Integer, primary_key=True),
                                  
    Column('users_id', Integer, ForeignKey('users.id'), primary_key=True),
)
shares_users_associations.__table_args__ = (ForeignKeyConstraint(
    ['shares_user', 'shares_group', 'shares_dataset'],
    ['shares.user_id', 'shares.group_id', 'shares.dataset_id'],),)

shares_groups_associations = Table('shares_groups_associations', Base.metadata,
    Column('shares_user', Integer, primary_key=True),
    Column('shares_group', Integer, primary_key=True),
    Column('shares_dataset', Integer, primary_key=True),
    Column('groups_id', Integer, ForeignKey('groups.id'), primary_key=True)
)
shares_groups_associations.__table_args__ = (ForeignKeyConstraint(
    ['shares_user', 'shares_group', 'shares_dataset'],
    ['shares.user_id', 'shares.group_id', 'shares.dataset_id'],),)

shares_datasets_associations = Table('shares_datasets_associations', Base.metadata,
    Column('shares_user', Integer, primary_key=True),
    Column('shares_group', Integer, primary_key=True),
    Column('shares_dataset', Integer, primary_key=True),
    Column('datasets_id', Integer, ForeignKey('datasets.id'), primary_key=True)
)
shares_datasets_associations.__table_args__ = (ForeignKeyConstraint(
    ['shares_user', 'shares_group', 'shares_dataset'],
    ['shares.user_id', 'shares.group_id', 'shares.dataset_id'],),)


class Share(Base, TableNameMixin, TimestampMixin):
    user_id = Column(Integer, ForeignKey('users.id'), primary_key=True)
    group_id = Column(Integer, ForeignKey('groups.id'), primary_key=True)
    dataset_id = Column(Integer, ForeignKey('datasets.id'), primary_key=True)

collection_enfaceimage_associations = Table('collection_enfaceimage_associations', Base.metadata, 
    Column('collections_id', Integer, ForeignKey('collections.id')),
    Column('enfaceimages_id', Integer, ForeignKey('enfaceimages.id'))
)

collection_volumeimage_associations = Table('collection_volumeimage_associations', Base.metadata, 
    Column('collections_id', Integer, ForeignKey('collections.id')),
    Column('volumeimages_id', Integer, ForeignKey('volumeimages.id'))
)

class Collection(Base, TableNameIdMixin):
    """A collection holds images of one Patient
    
    A collection holds all images of one Patient which should be evaluated together. 
    For a cross-sectional study it might contain images from a single visit
    while for a longitudinal study it can contain all images in a certain timeframe.
    Images in a collection should be registered.
    """
    registered = Column(Boolean, nullable=False, default=False)
    
    # Connection to patients table
    patient_id = Column(Integer, ForeignKey('patients.id'), nullable=False)
    patient = relationship("Patient", back_populates="collections")
    
    # Connection to datasets table
    dataset_id = Column(Integer, ForeignKey('datasets.id'), nullable=False)
    dataset = relationship("Dataset", back_populates = "collections")
    
    # Connections to image tables
    enfaceimages = relationship("EnfaceImage",
                          secondary=collection_enfaceimage_associations, 
                          back_populates = "collections")    
    volumeimages = relationship("VolumeImage",
                          secondary=collection_volumeimage_associations, 
                          back_populates = "collections")
    
groups_patients_associations = Table('groups_patients_associations', Base.metadata,
    Column('patients_id', Integer, ForeignKey('patients.id'), primary_key=True),
    Column('groups_id', Integer, ForeignKey('groups.id'), primary_key=True)
)
    
class Patient(Base, TableNameIdMixin):
    pseudonym = Column(String(30), nullable=False)
    firstname = Column(String(50), nullable=True)
    lastname = Column(String(50), nullable=True)
    gender = Column(Enum('M', 'F'))
    birthday = Column(Date, nullable=False)
    
    collections = relationship("Collection", back_populates="patient")
    groups = relationship("Group", secondary=groups_patients_associations, back_populates="patients")
    
    
    enfaceimages = relationship("EnfaceImage", back_populates="patient")
    volumeimages = relationship("VolumeImage", back_populates="patient")
    
    def __repr__(self):
        return "<Patient(firstname={}, lastname={}, id={}, gender={}, birthday={})>".format(
            self.firstname, self.lastname, self.id, self.gender, self.birthday)
    
class EnfaceImage(Base, TableNameIdMixin):
    patient_id = Column(Integer, ForeignKey('patients.id'), nullable=False)
    visit_date = Column(Date, nullable=False)
    
    filepath = Column(String(500),  nullable=False)
    
    collections = relationship("Collection",
                               secondary=collection_enfaceimage_associations, 
                               back_populates="enfaceimages" )
    patient = relationship("Patient", back_populates="enfaceimages")
    
    registered_volume = relationship("VolumeImage", back_populates="localizer_image")
    
    features = relationship("Feature", back_populates="enfaceimage")
    
    area_segmentations = None
    box_segmentations = None
    registration = None
    
class VolumeImage(Base, TableNameIdMixin):
    patient_id = Column(Integer, ForeignKey('patients.id'), nullable=False)
    visit_date = Column(Date, nullable=False)
    
    slices = relationship("Slice", back_populates="volumeimage")
    
    collections = relationship("Collection",
                               secondary=collection_volumeimage_associations, 
                               back_populates="volumeimages")
    patient = relationship("Patient", back_populates="volumeimages")
    
    localizer_id = Column(Integer, ForeignKey('enfaceimages.id'))
    localizer_image = relationship("EnfaceImage", back_populates="registered_volume", uselist=False)
    
    line_segmentations = None
    area_segmentations = None
    box_segmentations = None
    registration = None
    
class Slice(Base, TableNameIdMixin):
    start_x = Column(Integer, nullable=False)
    start_y = Column(Integer, nullable=False)
    
    end_x = Column(Integer, nullable=False)
    end_y = Column(Integer, nullable=False)
    
    volumeimage_id = Column(Integer, ForeignKey('volumeimages.id'))
    volumeimage = relationship("VolumeImage", back_populates = "slices")
    
    #line_segmentations = relationship()
    #area_segmentations = relationship()
    #box_segmentations = relationship()
    
    
class Feature(Base, TableNameIdMixin, TimestampMixin):
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    user = relationship("User")
    
    enfaceimage_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    enfaceimage = relationship("EnfaceImage", back_populates="features")
    
    pos_x = Column(Integer, nullable=False)
    ps_y = Column(Integer, nullable=False) 

featurematchs_affineregs_associations = Table('featurematchs_affineregs_associations',
    Base.metadata,
    Column('featurematchs_id', Integer, ForeignKey('featurematchs.id'), primary_key=True),
    Column('affineregs_id', Integer, ForeignKey('affineregistrations.id'), primary_key=True)
)

class FeatureMatch(Base, TableNameIdMixin, TimestampMixin):
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    user = relationship("User")
    
    feature1_id = Column(Integer, ForeignKey('features.id'), nullable=False)
    feature2_id = Column(Integer, ForeignKey('features.id'), nullable=False)
    feature1 = relationship("EnfaceImage", foreign_keys=[feature1_id])
    feature2 = relationship("EnfaceImage", foreign_keys=[feature2_id])
    
    affine_registrations = relationship("AffineRegistration",
                                        secondary=featurematchs_affineregs_associations,
                                        back_populates="feature_matchs")
    
class AffineRegistration(Base, TableNameIdMixin, TimestampMixin):
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    user = relationship("User")
    
    feature_matchs = relationship("FeatureMatch",
                                   secondary=featurematchs_affineregs_associations,
                                   back_populates="affine_registrations")
    parameters = Column(String(250))
    

In [ ]:
points_lines_associations = Table('points_lines_associations', Base.metadata,
    Column('points_id', Integer, ForeignKey('points.id'), primary_key=True),
    Column('lines_id', Integer, ForeignKey('linesegmentations.id'), primary_key=True)
)

class Point(Base, TableNameIdMixin):
    x_value = Column(Integer, nullable=False)
    y_value = Column(Integer, nullable=False)
    
class LineSegmentation(Base, TableNameIdMixin, AnnotationMixin):
    line_heigts = Column(String(8000))
    
    points = relationship("Point", secondary = points_lines_associations)
    linetype_id = Column(Integer ForeignKey("linetypes.id"))
    line_type = relationship("LineType")
    
class LineType(Base, TableNameIdMixin):
    region_acronym = Column(String(10), nullable=False)
    description = Column(String(400), nullable=False)
    position = Column(Enum("top", "bottom", "center"))    

class AreaSegmentation(Base, TableNameIdMixin, AnnotationMixin):
    area = Column(BLOB)
    top_left = Column(Integer, ForeignKey('points.id'))
    bot_right = Column(Integer, ForeignKey('points.id'))

class AreaType(Base, TableNameIdMixin):
    region_acronym = Column(String(10), nullable=False)
    description = Column(String(400), nullable=False)

class BoxSegmentation(Base, TableNameIdMixin, AnnotationMixin):
    top_left = Column(Integer, ForeignKey('points.id'))
    bot_right = Column(Integer, ForeignKey('points.id'))

class BoxType(Base, TableNameIdMixin):
    region_acronym = Column(String(10), nullable=False)
    description = Column(String(400), nullable=False)

In [5]:

    

"""
class QuadraticRegistration(Base, TableNameIdMixin, AnnotationMixin):
    fixed_id = Column(Integer, ForeignKey('enfaceimages.id'), nullable=False)
    moving_id = Column(Integer, ForeignKey('enfaceimages.id'), nullable=False)
    
    fixed = relationship("EnfaceImage", back_populates="registrations")
    moving = relationship("EnfaceImage", back_populates="registrations")
    
    matches = relationship("PointMatch",
                           secondary=None
                           back_populates="registrations")
    
    parameters = Column(ARRAY(Integer))
    
class DeformableRegistration(Base, TableNameIdMixin, AnnotationMixin):
    fixed_id = Column(Integer, ForeignKey('enfaceimages.id'), nullable=False)
    moving_id = Column(Integer, ForeignKey('enfaceimages.id'), nullable=False)
    
    fixed = relationship("EnfaceImage", back_populates="registrations")
    moving = relationship("EnfaceImage", back_populates="registrations")
    
    matches = relationship("PointMatch",
                           secondary=None
                           back_populates="registrations")
    filepath = Column(String(200))
"""

In [5]:
Base.metadata.create_all(engine)

2020-03-12 17:47:09,149 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-12 17:47:09,151 INFO sqlalchemy.engine.base.Engine ()
2020-03-12 17:47:09,154 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-12 17:47:09,155 INFO sqlalchemy.engine.base.Engine ()
2020-03-12 17:47:09,159 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users_groups_associations")
2020-03-12 17:47:09,160 INFO sqlalchemy.engine.base.Engine ()
2020-03-12 17:47:09,163 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users_groups_associations")
2020-03-12 17:47:09,163 INFO sqlalchemy.engine.base.Engine ()
2020-03-12 17:47:09,164 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-03-12 17:47:09,165 INFO sqlalchemy.engine.base.Engine ()
2020-03-12 17:47:09,166 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-03-12 17:47:09,166 INFO sqlalchemy.engine.b

In [7]:
def create_user(fname, lname, uname, pw, session):
    new_user = User(firstname=fname, lastname=lname, username=uname, password_hash=hash_password(pw))
    
    for groupname in [uname, "all"]:
        add_user_to_group(new_user, groupname)
        
    session.add(new_user)
    session.commit()
    
def add_user_to_group(user, groupname):
    query = session.query(Group).filter(Group.name == groupname)
    
    try:
        group = query.one()
        user.groups.append(group)
        
    except NoResultFound:
        user.groups.append(Group(name=groupname))
        
def create_dataset(name, info, groups):
    new_dataset = Dataset(name=name, info=info)
    
def add_visit_to_dataset():
    pass
        
    

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

create_user("Donald", "Duck", "donald", "quack", session)
create_user("Max", "Mustermann", "max", "test", session)

In [ ]:
query = session.query("groups").filter(Groups.name == group_name)
query.one()

In [6]:

new_user = User(firstname="Max", lastname="Mustermann", username="max", password_hash=hash_password("test"))
new_user.groups = [Group(name="max"), Group(name="all")]
session.add(new_user)
session.commit()

In [ ]:
def create_user(fname, lname, uname, pw):
    new_user = User(firstname=fname, lastname=lname, username=uname, password_hash=hash_password(pw))

In [ ]:
class Patient(Base, TableNameIdMixin, PersonMixin):
    gender = Column(Enum('M', 'F'))
    birthday = Column(Date, nullable=False)
    
    def __repr__(self):
        return "<Patient(firstname={}, lastname={}, id={}, gender={}, birthday={})>".format(
            self.firstname, self.lastname, self.id, self.gender, self.birthday)
    
class Visit(Base, TableNameIdMixin):
    patient_id = Column(Integer, ForeignKey('patients.id'), nullable=False)
    visit_date = Column(Date, nullable=False)    

class Image(Base, TableNameIdMixin):
    patient_id = Column(Integer, ForeignKey('patients.id'), nullable=False)
    visit_id = Column(Integer, ForeignKey('visits.id'), nullable=False)
    filepath = Column(String(500),  nullable=False)
    
class ImageMeta(Base, TableNameIdMixin):
    modality = Column(Enum('CFP', 'OCT', 'NIR'),  nullable=False)
    eye = Column(Enum('Left', 'Right'),  nullable=False)
    image_id = None
    
class Registration(Base, TableNameIdMixin):
    image1_id = Column(Integer, ForeignKey('images.id'), nullable=False)
    image2_id = Column(Integer, ForeignKey('images.id'), nullable=False)
    registration_type = None
    parameters = None
    
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)    
    user = relationship("User", foreign_keys=[user_id])
    
class Correspondence(Base, TableNameIdMixin, TimestampMixin):
    registration_id = Column(Integer, ForeignKey('registrations.id'), nullable=False)
    point1_id = Column(Integer, ForeignKey('points.id'), nullable=False)
    point2_id = Column(Integer, ForeignKey('points.id'), nullable=False)
    
    
    registration = relationship("Registration", foreign_keys=[registration_id])
    point1 = relationship("Point", foreign_keys=[point1_id])
    point2 = relationship("Point", foreign_keys=[point2_id])
    

class Point(Base, TableNameIdMixin)
    xvalue = Column(Integer, nullable=False)
    yvalue = Column(Integer, nullable=False)
    
    

class LineSegmentation(Base, TableNameIdMixin):
    image_id = None
    
    # RPE, HRF, Fovea ...
    region = None
    
    # Guideline 
    guideline = None    
    
class AreaSegmentation(Base, TableNameIdMixin):
    pass
    
class Shapesegmentation(Base, TableNameIdMixin):
    pass
    
    
    



    